## Handling the the data of Family.csv

### Pre-processing

Load Data

In [9]:
import pandas as pd
import numpy as np
import re

data = pd.read_csv('FamilyData.csv')
dt = data

Check for duplication

In [10]:
duplicates = dt.duplicated()
print("Duplicates:")
print(dt[duplicates])

Duplicates:
Empty DataFrame
Columns: [Family, Member, Income, Spend]
Index: []


- There is no duplicates.

Check for empty values

In [11]:
empty_values = dt.isnull().any(axis=1)
print("\nEmpty values:")
print(dt[empty_values])


Empty values:
Empty DataFrame
Columns: [Family, Member, Income, Spend]
Index: []


- There is no empty data frame

### Prepocessing 

### A1 Total Income

The dataset is built based on each person. It's easier for further process after transforming it into family base.

Define "Family_num" : a new column contains only number of the family.
Accomplised by extracting the number in "Family" column.

In [12]:
def family_num(x):
    num = re.findall(r'\d+', x)
    return int(num[0])
dt["Family_num"] = dt["Family"].apply(family_num)
dt["Family_num"] = dt["Family_num"].astype(int)

Calculating the total number of adults and chuldren in each family

In [13]:
dt["Adult_num"] = dt["Member"].str.contains("Adult")
dt["Child_num"] = dt["Member"].str.contains("Child")

Aggregate data into family based.

In [14]:
def join_member(x):
    return " ".join(x)
aggregation_function = {"Spend":"sum","Income":"sum","Family_num":"first","Adult_num":"sum","Child_num":"sum","Member":join_member}
dt = dt.groupby(dt["Family"]).aggregate(aggregation_function)
dt["Family_num"] = dt["Family_num"].astype(int)
dt = dt.sort_values(by = ["Family_num"], ascending = True)
dt

,Spend,Income,Family_num,Adult_num,Child_num,Member
Family,,,,,,
family1,2129097,4761087,1,3,0,Adult1 Adult2 Adult3
family2,890424,2939887,2,3,2,Adult1 Adult2 Adult3 Child1 Child2
family3,807835,2301931,3,1,0,Adult1
family4,1128708,2896133,4,2,0,Adult1 Adult2
family5,501827,1428679,5,3,1,Adult1 Adult2 Adult3 Child1
...,...,...,...,...,...,...
family96,135954,325062,96,1,2,Adult1 Child1 Child2
family97,774694,2663794,97,2,0,Adult1 Adult2
family98,1031955,3018609,98,2,0,Adult1 Adult2


Family that has the highest total income.

In [15]:
print ("Highest annual income: ")
print(dt.loc[dt["Income"].idxmax()])


Highest annual income: 
Spend                                    2879221
Income                                   7804425
Family_num                                     6
Adult_num                                      3
Child_num                                      2
Member        Adult1 Adult2 Adult3 Child1 Child2
Name: family6, dtype: object


Family that has the lowest total income.

In [16]:
print ("Lowest annual income: ")
print(dt.loc[dt["Income"].idxmin()])

Lowest annual income: 
Spend                 30029
Income                46790
Family_num               94
Adult_num                 1
Child_num                 1
Member        Adult1 Child1
Name: family94, dtype: object


### A2 Possess adequate annual income

Check if there is a family doesn't have enough incomes for spending

In [17]:
print("Spending exceed income: ",dt[dt["Spend"] > dt["Income"]])


Spending exceed income:  Empty DataFrame
Columns: [Spend, Income, Family_num, Adult_num, Child_num, Member]
Index: []


As the result, every family possess enough incomes for spending

### A3-1 Single Parent

I see single family as a family with at least 1 child but only 1 parent at present

In [18]:
dt_single = dt.loc[(dt['Adult_num'] == 1) & (dt['Child_num'] > 0)]
print (dt_single)
print ("Single parent family: ",dt_single['Family_num'].count())

            Spend   Income  Family_num  Adult_num  Child_num  \
Family                                                         
family7   1298626  2815513           7          1          1   
family14   261304   998732          14          1          2   
family15    62395   156330          15          1          2   
family21  1499402  2979034          21          1          2   
family22   625138  2559448          22          1          1   
family27   149986   459792          27          1          1   
family33    91991   331955          33          1          2   
family37   710857  2924516          37          1          1   
family45   871635  2477184          45          1          1   
family48    74349   198274          48          1          1   
family54   497764  1317286          54          1          1   
family59   492122  1458533          59          1          1   
family61   790274  1680099          61          1          2   
family64   185343   770201          64  

### A3-2 Childless

For childless families, I take the families with no children and the number of adults is less than 3.

In [19]:
dt_childless = dt.loc[((dt['Child_num'] == 0) & (dt['Adult_num'] < 3)) ]
print (dt_childless)
print ("Childless family: ",dt_childless['Family_num'].count())


             Spend   Income  Family_num  Adult_num  Child_num         Member
Family                                                                      
family3     807835  2301931           3          1          0         Adult1
family4    1128708  2896133           4          2          0  Adult1 Adult2
family9    2115220  4653044           9          2          0  Adult1 Adult2
family13   1767505  5493031          13          2          0  Adult1 Adult2
family17    222356   880849          17          2          0  Adult1 Adult2
family24   1551899  3806938          24          2          0  Adult1 Adult2
family25    190368   422690          25          1          0         Adult1
family32    457300  2028187          32          1          0         Adult1
family36    126095   561059          36          2          0  Adult1 Adult2
family38   1197432  2497154          38          1          0         Adult1
family42    911170  2155619          42          1          0         Adult1